## Homework

In [509]:
#!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [510]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_blobs, make_circles, make_classification, load_iris, load_digits
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import tree, metrics, model_selection
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from collections import Counter

In [511]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])

Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [512]:
df = df.dropna()

In [513]:
X = df.drop('price', axis=1)
y = df['price']

In [514]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

In [515]:
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]

cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]

target_column = ['price']

num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns + target_column)))

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [516]:
N = 50

In [517]:
score0 = np.zeros(N)

for n in range(N):
    baseline_model = DecisionTreeRegressor().fit(X_train[num_columns], y_train)
    score0[n] = metrics.mean_squared_error(y_test, baseline_model.predict(X_test[num_columns]))

Проверьте качество на отложенной выборке.

In [518]:
score0.mean()

24514184052482.855

### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [719]:
X = df.drop('price', axis=1)
y = df['price']

In [720]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
X["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
X["week_year_cnt"] = week_year.map(week_year_cnt_map)

/var/folders/9j/cllsf0815jl9bm0yjv_8cm400000gn/T/ipykernel_8573/3259660219.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [721]:
X['month'] = X.timestamp.dt.month
X['week_day'] = X.timestamp.dt.weekday

X['floor_max_floor'] = X['floor'] / X['max_floor']
X.loc[X['floor_max_floor'] == np.inf, 'floor_max_floor'] = np.NaN
X['floor_max_floor'] = X['floor_max_floor'].fillna(X['floor_max_floor'][X['floor_max_floor'].notna()].mean())

X['kitch_full'] = X['kitch_sq'] / X['full_sq']
X.loc[X['kitch_full'] == np.inf, 'kitch_full'] = np.NaN
X['kitch_full'] = X['kitch_full'].fillna(X['kitch_full'][X['kitch_full'].notna()].mean())

In [722]:
X['culture_objects_top_25_int'] = (X['culture_objects_top_25'] == 'yes')
X['thermal_power_plant_int'] = (X['thermal_power_plant_raion'] == 'yes')
X['incineration_raion_int'] = (X['incineration_raion'] == 'yes')
X['oil_chemistry_raion_int'] = (X['oil_chemistry_raion'] == 'yes')
X['radiation_raion_int'] = (X['radiation_raion'] == 'yes')
X['railroad_terminal_raion_int'] = (X['railroad_terminal_raion'] == 'yes')
X['big_market_raion_int'] = (X['big_market_raion'] == 'yes')
X['nuclear_reactor_raion_int'] = (X['nuclear_reactor_raion'] == 'yes')
X['detention_facility_raion_int'] = (X['detention_facility_raion'] == 'yes')
X['product_type_int'] = (X['product_type'] == 'yes')

In [723]:
try:
    X = X.drop(['kitch_sq'], axis=1)
except KeyError:
    print('already deleted')

Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [724]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [725]:
num_columns = list(set(X.columns).difference(set(cat_columns + drop_columns + target_column)))

In [711]:
score1 = np.zeros(N)

for n in range(N):
    model1 = DecisionTreeRegressor().fit(X_train[num_columns], y_train)
    score1[n] = metrics.mean_squared_error(y_test, model1.predict(X_test[num_columns]))

In [712]:
print(score1.mean() / score0.mean())

0.9671290813151279


### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [730]:
from sklearn.model_selection import GridSearchCV

search = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid={'min_samples_split': range(1, 10)}, scoring=metrics.mean_squared_error, sco)

search.fit(X_train[num_columns], y_train, X)

/opt/homebrew/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: mean_squared_error() takes 2 positional arguments but 3 were given

  warnings.warn(
/opt/homebrew/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: mean_squared_error() takes 2 positional arguments but 3 were given

  warnings.warn(
/opt/homebre

ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).